# **SpaceX  Falcon 9 First Stage Landing Prediction**

## EDA with SQL(SQLITE3) :

## Overview of the Dataset

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. 


Therefore, if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


In [67]:
# Install the necessary packages to use within this part of the project using pip

!pip install sqlalchemy==1.3.9  # ORM for databases
!pip install -q pandas==1.1.5
!pip install ipython-sql  # SQL magic function
!pip install --upgrade ipython  # update to SQL magic function

### Connect to the database

Let's first load the SQL extension and establish a connection with the database.

In [14]:
# We execute this command so we could use SQL magic commands (Loading SQL extension)

%load_ext sql

In [26]:
import csv, sqlite3

con = sqlite3.connect("spacex_data.sqlite")
cur = con.cursor()

In [68]:
# Connecting to database using SQL magic function

%sql sqlite:///spacex_data.sqlite

### Download the dataset

We need to load the spacex dataset and save it into a sqlite3 database. It's available on IBM cloud gallery in the format of a csv file. It's a better version than the one we imported using SpaceX public API.

Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>

In [28]:
import pandas as pd

df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

## SQL Queries


### Query 1




##### The names of the unique launch sites  in the space mission :

In [30]:
# %sql select distinct launch_site from SPACEXTBL  # using SQL magic function

res = cur.execute("select distinct launch_site from SPACEXTBL")
res_df = pd.DataFrame(res.fetchall(), columns=['Launch_Site'])
res_df

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40



### Query 2


#####  The first 5 records where launch sites begin with the string 'CCA'  :

In [31]:
# %sql select * from spacextbl where launch_site like 'CCA%' limit 5  # using SQL magic function

res = cur.execute("select * from spacextbl where launch_site like 'CCA%' limit 5")
res_df = pd.DataFrame(res.fetchall(), columns=['Date', 'Time', 'Booster_Version', 'Launch_Site', 'Payload', 'Payload_Mass_Kg', 'Orbit', 'Customer', 'Mission_Outcome', 'Landing_Outcome'])
res_df

,Date,Time,Booster_Version,Launch_Site,Payload,Payload_Mass_Kg,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Query 3




##### The total payload mass carried by boosters launched by NASA (CRS) :

In [35]:
# %sql select sum(PAYLOAD_MASS__KG_) as SUM from spacextbl where customer = 'NASA (CRS)'  # using SQL magic function

res = con.execute("select sum(PAYLOAD_MASS__KG_) as SUM from spacextbl where customer = 'NASA (CRS)'")
print("The total payload mass carried by boosters launched by NASA (CRS) is", res.fetchall()[0][0], "Kg")

The total payload mass carried by boosters launched by NASA (CRS) is 45596 Kg


### Query 4




##### The average payload mass carried by booster version F9 v1.1 :

In [36]:
# %sql select avg(PAYLOAD_MASS__KG_) as AVG from spacextbl where booster_version like 'F9 v1.1'  # using SQL magic function

res = con.execute("select avg(PAYLOAD_MASS__KG_) as AVG from spacextbl where booster_version like 'F9 v1.1'")
print("The average payload mass carried by booster version F9 v1.1 is", res.fetchall()[0][0], "Kg")

The average payload mass carried by booster version F9 v1.1 is 2928.4 Kg


### Query 5

##### The date when the first succesful landing outcome in ground pad was acheived :

In [45]:
# %sql select min(date) as min from spacextbl where Landing_Outcome = 'Success (ground pad)'  # using SQL magic function

res = con.execute("select min(date) as min from spacextbl where Landing_Outcome = 'Success (ground pad)'")
print("The date when the first successful landing outcome in ground pad was achieved is", res.fetchall()[0][0])

The date when the first successful landing outcome in ground pad was achieved is 2015-12-22


### Query 6

##### The names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000 :

In [46]:
# %sql select booster_version from spacextbl where Landing_Outcome = 'Success (drone ship)' and (payload_mass__kg_ > 4000 and payload_mass__kg_ < 6000)  # using SQL magic function

res = con.execute("select booster_version from spacextbl where Landing_Outcome = 'Success (drone ship)' and (payload_mass__kg_ > 4000 and payload_mass__kg_ < 6000)")
res_df = pd.DataFrame(res.fetchall(), columns=['Booster_Version'])
res_df

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### Query 7




##### The total number of successful and failure mission outcomes :

In [47]:
# %sql select Mission_Outcome, count(*) as Count_Outcomes from spacextbl group by Mission_Outcome  # using SQL magic function

res = con.execute("select Mission_Outcome, count(*) as Count_Outcomes from spacextbl group by Mission_Outcome")
res_df = pd.DataFrame(res.fetchall(), columns=['Mission_Outcome', 'Count_Outcomes'])
res_df

,Mission_Outcome,Count_Outcomes
0,Failure (in flight),1
1,Success,98
2,Success,1
3,Success (payload status unclear),1


So, we have 99 successful missions , one failure and a doubtful mission outcome (payload status unclear).

### Query 8



##### The names of the booster_versions which have carried the maximum payload mass :

In [48]:
# %sql select booster_version from spacextbl where payload_mass__kg_ = (select max(payload_mass__kg_) from spacextbl)  #using SQL magic function

res = con.execute("select booster_version from spacextbl where payload_mass__kg_ = (select max(payload_mass__kg_) from spacextbl)")
res_df = pd.DataFrame(res.fetchall(), columns=['Booster_Version'])
res_df

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


### Query 9

**Note: SQLLite does not support monthnames. So we need to use substr(Date, 6, 2) as month to get the months and substr(Date,1,4)='2015' for year.**

##### The records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015 :

In [54]:
# %sql select substr(date, 4, 2) as month, Landing_Outcome, booster_version, launch_site from spacextbl where substr(date, 7, 4)='2015' and Landing_Outcome='Failure (drone ship)'  # using SQL magic function

res = con.execute("select substr(date, 6, 2) as month, Landing_Outcome, Booster_Version, Launch_Site from spacextbl where substr(date, 1, 4) = '2015' and Landing_Outcome = 'Failure (drone ship)'")
res_df = pd.DataFrame(res.fetchall(), columns=['Month', 'Landing_Outcome', 'Booster_Version', 'Launch_Site'])
res_df

,Month,Landing_Outcome,Booster_Version,Launch_Site
0,10,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
1,04,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### Query 10




##### Ranking the  count of  successful landing_outcomes between the date 04-06-2010 and 20-03-2017 in descending order :

In [65]:
# %sql select Landing_Outcome, count(*) as count from spacextbl where Date between '04-06-2010' and '20-03-2017' group by Landing_Outcome having Landing_Outcome like '%Success%' order by count desc  # using SQL magic function

res = con.execute("select Landing_Outcome, count(*) as Count_Outcomes from spacextbl where date between '2010-06-04' and '2017-03-20' group by Landing_Outcome having (Landing_Outcome like '%Success%') order by 2 desc")
res_df = pd.DataFrame(res.fetchall(), columns=['Landing_Outcome', 'Count_Outcomes'])
res_df

,Landing_Outcome,Count_Outcomes
0,Success (ground pad),5
1,Success (drone ship),5


In [66]:
# Closing the connection with our newly created database

con.close()